### Why Plotly?

Plotly provides an alternative to `matplotlib` with additional interactivity features and the option to save your figures as .html files. This allows the user to share interactive graphs without providing any of the code. Plotly is available for
- Python
- Julia
- R
- Matlab
- F#

### Two design philosophies

Matplotlib already had two different design philosophies with the straighforward approach, i.e. simply using functions `plt.plot()`, and the object oriented approach using the `axes` class, 
````python
ax = plt.axes()
ax.plot([1,2,3])
````

Plotly's design is characterized by two packages. `plotly.graph_objs` contains the full functionality of plotly. However, it comes with a bit of a learning curve. `plotly.express` was later introduced with simplified syntax but reduced functionality.

### `plotly.express`

In [ ]:
import plotly.express as px
import numpy as np

x = np.linspace(0,2*np.pi,1000)
y = np.sin(x) + np.random.normal(0,0.1,1000)

fig = px.scatter(x=x, y=y)
fig.show()

In [ ]:
fig = px.line(x=x, y=y)
fig.show()

One of the ideas behind `plotly.express` is to plot your data directly from a given dictionary/dataframe

In [ ]:
import pandas as pd
data = pd.read_csv('CellData.csv')
data.head()

In [ ]:
px.scatter(data, x='x-Position', y='y-Position', color='Type')

### `plotly.graph_objs`

The above plot is created in a single line of code. It includes labels, legend and different colors. However, customizing a plot further often requires the use of `plotly.graph_objs` which is no longer based on dictionaries/dataframes. The same plot now requires more steps. The following code only creates the plot without any additional information:

In [ ]:
import plotly.graph_objs as go

fig= go.Figure()
fig.add_trace(
              go.Scatter(x=data['x-Position'], y=data['y-Position'], mode='markers')
)
fig.show()

If we want different colors, we need to assign colors to every marker.

In [ ]:
colors = ['blue' if typ == 'N-G+' else 'red' for typ in data['Type']]

fig= go.Figure()
fig.add_trace(
              go.Scatter(x=data['x-Position'], y=data['y-Position'], mode='markers',
                         marker={'color': colors})
)
fig.show()

Now we have colors, but there is still no legend, labels and hoverinfo besides the actual data. Every single type of information has to be added manually. This can often be quite tedious, see example:

In [ ]:
info = '%{hovertext}<br>x-Position = %{x}<br>y-Position = %{y}<extra></extra>'
N = data[data['Type'] == 'N+G-']
G = data[data['Type'] == 'N-G+']
            
fig= go.Figure()
fig.add_trace(
              go.Scatter(x=N['x-Position'], y=N['y-Position'], mode='markers',
                         marker = {'color': ['red']*len(N)},
                         hovertemplate = info,
                         hovertext = N['Type'],
                         name = 'N+G-'
                        )
)
fig.add_trace(
              go.Scatter(x=G['x-Position'], y=G['y-Position'], mode='markers',
                         marker = {'color': ['blue']*len(G)},
                         hovertemplate = info,
                         hovertext = N['Type'],
                         name = 'N-G+'
                        )
)
fig.update_layout({'xaxis':{'title_text':'x-Position'},
                   'yaxis':{'title_text':'y-Position'}}
)
fig.show()

As demonstrated, every part of the legend has to be added as a separate trace in our graph object. Remains the question why one should use `plotly.graph_objs` instead of `plotly.express`. One example might be subploting which is currently not supported by `plotly.express`. A full example of how such a plot might look like is:

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

# Some data do plot
x = np.linspace(0,1,100)
y1 = x
y2 = x**2
y3 = x**(1/2)
y4 = x**(1/3)
pos = np.random.uniform(10,20,[100,3])

# Color scheme and info for bar plot
types = ['DP & DN', 'N+G-', 'N-G-']
props = [30, 50, 20]
colors =['rgba(0.75,0.75,0.75,1)', 'rgba(0.75,0,0.75,1)', 'rgba(0,0.75,0.75,1)']

# Initialize subplots
fig = make_subplots(rows=2, cols=3,specs=[[{}, {"rowspan": 2, "colspan": 2, "type": "scatter3d"}, None],
                                          [{}, None, None]])

# Hoverinfo for first plot
infopcf1 = '<b>PCF maximum (N+G-)</b><br>dist = %{x}<br>PCF = %{y}<extra></extra>'
infopcf2 = '<b>PCF minimum (N+G-)</b><br>dist = %{x}<br>PCF = %{y}<extra></extra>'
infopcf3 = '<b>PCF minimum (N-G+)</b><br>dist = %{x}<br>PCF = %{y}<extra></extra>'
infopcf4 = '<b>PCF maximum (N-G+)</b><br>dist = %{x}<br>PCF = %{y}<extra></extra>'

# Add 4 plots to the upper left corner
fig.add_trace(
    go.Scatter(x=x, y=y1, fill=None, mode='lines', line_color='rgba(0.75,0,0.75,1)', hovertemplate=infopcf1),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=x, y=y2, fill='tonexty',mode='lines', line_color='rgba(0.75,0,0.75,1)', hovertemplate=infopcf2),
    row=1, col=1 
)
fig.add_trace(
    go.Scatter(x=x, y=y3, fill=None, mode='lines', line_color='rgba(0,0.75,0.75,1)', hovertemplate=infopcf3),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=x, y=y4, fill='tonexty',mode='lines', line_color='rgba(0,0.75,0.75,1)', hovertemplate=infopcf4),
    row=1, col=1 
)

# Add 3 bars to the lower left corner
for i in range(3):
    fig.add_trace(go.Bar(x=[types[i]], y = [props[i]], name=types[i], marker_color=colors[i]), row=2, col=1)

# Define color scheme and info for each point in scatterplot
test = []
text = []
for i in range(len(pos)):
    if i%3 == 0:
        test.append('rgba(0.75,0.75,0.75,1)')
        text.append('<b>DN or DP</b>')
    if i%3 == 1:
        test.append('rgba(0.75,0,0.75,1)')
        text.append('<b>N+G-</b>')
    if i%3 == 2:
        test.append('rgba(0,0.75,0.75,1)')
        text.append('<b>N-G+</b>')

# Define hovertext template for each marker
info3d = '%{hovertext}<br>x = %{x}<br>y = %{y}<br>z = %{z}<extra></extra>'

# Add 3D scatterplot to the right
fig.add_trace(
    go.Scatter3d(x=pos[:,0],y=pos[:,1],z=pos[:,2], mode='markers', hovertext = text, hovertemplate = info3d,
    marker=dict(
    size=10,
    color=test,
    line=dict(
        color='black',
        width=1
    )
    )),
    row=1, col=2
)

# Increase figure size, include title and set background color
fig.update_layout(height=600, width=900, title_text="Title", paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)',showlegend=False)

# Add x- and y- axes
fig.update_layout({'xaxis':{'title_text':'distances','linecolor':'black'},
                   'yaxis':{'title_text':'$\\rho$','linecolor':'black'},
                   'xaxis2':{'title_text':'cell type','linecolor':'black'},
                   'yaxis2':{'range': [0, 100],'dtick': 20, 'title_text':'proportions','linecolor':'black'},}
)

# Remove background and coordinate system in 3D plot
fig.update_layout(scene = dict(xaxis = dict(showbackground=False,showticklabels=False,visible=False),
                               yaxis = dict(showbackground=False,showticklabels=False,visible=False),
                               zaxis = dict(showbackground=False,showticklabels=False,visible=False)))

#fig.write_html("test.html")
fig.show()